In [11]:
import numpy as np

def spring_weight(x):
    D, d, N = x
    N = int(round(N))
    if D < 0.05 or D > 2 or d < 0.001 or d > 0.1 or N < 3 or N > 15:
        return 1e9
    weight = (N + 2) * D**2 * d
    if np.iscomplex(weight):
        print(f"Complex weight! D={D}, d={d}, N={N}, weight={weight}")
    return weight.real

def cuckoo_search(n=15, pa=0.25, iterations=100):
    nests = np.random.rand(n, 3)
    bounds = np.array([[0.05, 2.0], [0.001, 0.1], [3, 15]])
    for i in range(3):
        nests[:, i] = bounds[i, 0] + nests[:, i] * (bounds[i, 1] - bounds[i, 0])
    nests[:, 2] = np.round(nests[:, 2])
    nests = nests.astype(float)

    fitness = np.array([spring_weight(x) for x in nests])
    best = nests[np.argmin(fitness)]

    for it in range(iterations):
        new_nests = nests + np.random.randn(*nests.shape) * 0.1
        new_nests = new_nests.astype(float)
        
        for i in range(3):
            new_nests[:, i] = np.clip(new_nests[:, i], bounds[i, 0], bounds[i, 1])
        new_nests[:, 2] = np.round(new_nests[:, 2])

        if np.iscomplexobj(new_nests) or np.iscomplexobj(nests):
            print("Complex numbers detected in nests arrays!")

        new_fitness = np.array([spring_weight(x) for x in new_nests])

        for i in range(n):
            j = np.random.randint(0, n)
            if new_fitness[i] < fitness[j]:
                nests[j] = new_nests[i]
                fitness[j] = new_fitness[i]

        abandon = np.random.rand(n) < pa
        for i in range(3):
            nests[abandon, i] = bounds[i, 0] + np.random.rand(np.sum(abandon)) * (bounds[i, 1] - bounds[i, 0])
        nests[abandon, 2] = np.round(nests[abandon, 2])

        fitness[abandon] = [spring_weight(x) for x in nests[abandon]]

        best = nests[np.argmin(fitness)]

    return best, spring_weight(best)

best, best_val = cuckoo_search()
print("Best Design Parameters (D, d, N):", best)
print("Minimum Spring Weight:", best_val)



Best Design Parameters (D, d, N): [5.e-02 1.e-03 8.e+00]
Minimum Spring Weight: 2.5000000000000005e-05
